In [ ]:
import pandas as pd
import numpy as np
import os

# 假設資料已經事先整理好
# 原始資料請至 AI CUP 官方網站下載，並自行放置於指定資料夾

folder_path = 'your_data_path'  # 請使用者自行指定資料夾
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# 示範合併多個 CSV 檔案為單一 DataFrame
dataframes = [pd.read_csv(file) for file in csv_files]
df = pd.concat(dataframes, ignore_index=True)

In [ ]:
# 查看每列缺失值数量
print(df.isnull().sum())

In [ ]:
df.describe()

In [ ]:
# 轉換日期格式並提取時間特徵
df['DateTime'] = pd.to_datetime(df['DateTime'])
df['Hour'] = df['DateTime'].dt.hour
df['DayOfWeek'] = df['DateTime'].dt.weekday
df['Month'] = df['DateTime'].dt.month
df['Day'] = df['DateTime'].dt.day
df['Minute'] = df['DateTime'].dt.minute
df['sin_hour'] = np.sin(2 * np.pi * df['Hour'] / 24)
df['cos_hour'] = np.cos(2 * np.pi * df['Hour'] / 24)
df['sin_day_of_week'] = np.sin(2 * np.pi * df['DayOfWeek'] / 7)
df['cos_day_of_week'] = np.cos(2 * np.pi * df['DayOfWeek'] / 7)
df['sin_month'] = np.sin(2 * np.pi * df['Month'] / 12)
df['cos_month'] = np.cos(2 * np.pi * df['Month'] / 12)

In [ ]:
# 定義季節
def get_season(month):
    if month in [3, 4, 5]:
        return '0'
    elif month in [6, 7, 8]:
        return '1'
    elif month in [9, 10, 11]:
        return '2'
    else:
        return '3'

# 根據月份來創建季節特徵
df['Season'] = df['Month'].apply(get_season)

# 定義清晨、早晨、中午、下午、傍晚、深夜
def get_time(hour):
    if hour in [5, 6, 7]:
        return '0'
    elif hour in [8, 9, 10]:
        return '1'
    elif hour in [11, 12, 13]:
        return '2'
    elif hour in [14, 15, 16]:
        return '3'
    elif hour in [17, 18, 19]:
        return '4'
    else:
        return '5'

# 根據月份來創建季節特徵
df['Time'] = df['Hour'].apply(get_time)

## **資料集修正**

### **風速修正** 線性插值和均值插補

In [ ]:
import pandas as pd
import numpy as np

def fix_wind_speed(df):
    # 步驟 1: 使用線性插值填補風速為 0 或 NaN 的值
    df['WindSpeed(m/s)'] = df['WindSpeed(m/s)'].replace(0, np.nan)  # 替換 0 值為 NaN，方便插值
    df['WindSpeed(m/s)'] = df['WindSpeed(m/s)'].interpolate(method='linear')  # 使用線性插值填補

    # 步驟 2: 若仍有缺失值（NaN），使用均值填補
    df['WindSpeed(m/s)'].fillna(df['WindSpeed(m/s)'].mean(), inplace=True)

    return df

# 呼叫修正函數
df = fix_wind_speed(df)

<ipython-input-34-6bb635638644>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['WindSpeed(m/s)'].fillna(df['WindSpeed(m/s)'].mean(), inplace=True)


In [ ]:
df

### **光照度修正**

通過回歸分析，計算光照度與發電量之間的比率。
對達到最大值（117758.2 lux）的紀錄，通過回推發電量反推出光照度。
處理方法：

建立 Sunlight(Lux) 和 Power(mW) 的回歸模型。
用回歸模型對超過最大值的光照度進行估算修正。

In [ ]:
# 定義最大光照度
max_sunlight = 117758.2

# 找出光照度大於最大值的數據
exceed_max_sunlight = df[df['Sunlight(Lux)'] >= max_sunlight]

# 顯示大於最大值的數據筆數
print(f"Number of records with Sunlight(Lux) greater than {max_sunlight}: {exceed_max_sunlight.shape[0]}")


In [ ]:
# 模型訓練與視覺化（基於比賽資料，本範例省略實際輸出）
regressor = LinearRegression()
regressor.fit(X, y)

# 為遵守資料保密規範，以下為註解範例格式
# print(f"回歸係數: {regressor.coef_[0]}")
# print(f"截距: {regressor.intercept_}")

# 視覺化示意（圖形輸出已移除）
plt.scatter(X, y, color='blue', label='樣本點')
plt.plot(X, regressor.predict(X), color='red', label='回歸線')
plt.xlabel('Sunlight(Lux)')
plt.ylabel('Power(mW)')
plt.title('Sunlight vs Power (Linear Regression)')
plt.legend()
# plt.show()  # ← 示意用途，實際圖表未列出以保護比賽資料

In [ ]:
# 假設最大發電量
max_power = df['Power(mW)'].max()

# 根據回歸模型推算出對應的光照度
predicted_sunlight = (max_power - regressor.intercept_) / regressor.coef_[0]

print(f"基於最大發電量 {max_power} 推算的光照度為: {predicted_sunlight}")

In [ ]:
# 對大於最大值的光照度數據進行修正
df.loc[df['Sunlight(Lux)'] >= max_sunlight, 'Sunlight(Lux)'] = predicted_sunlight

# 顯示修正後的數據
print(df[['Sunlight(Lux)']].head())

In [ ]:
# 將處理後的資料輸出為 CSV（使用者自行指定輸出路徑）
df.to_csv('your_output_path/processed_data.csv', index=False)


# 上面為處理資料集的程式碼，已將處理後的存成csv，直接以處理好的訓練

In [ ]:
import pandas as pd
import numpy as np

# 如在 Google Colab 使用，掛載 Google Drive（選用）
# from google.colab import drive
# drive.mount('/content/drive')

# 匯入處理後資料（請替換為實際路徑）
df = pd.read_csv('your_data_path/processed_data.csv')

# 檢查資料讀取（為保護比賽資料結構，此處不輸出內容）
# print(df.head())

In [ ]:
df

In [ ]:
# 檢查資料欄位名稱（此處省略實際輸出以保護比賽資料欄位結構）
df.columns

### 相關性分析

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 指定特徵進行相關性分析
features = ['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)','Sunlight(Lux)',
            'Month','Day','Hour','Minute','LocationCode','Power(mW)']

correlation_matrix = df[features].corr()
print(correlation_matrix)

# 绘制热图
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

## 特徵間相關性分析摘要（依據訓練資料）

- 風速與地點有負相關，並與月份呈現一定的週期性關係。
- 氣壓與月份亦有正相關，並受到地點影響。
- 溫度與光照度、發電量皆呈現強正相關，與濕度則為負相關。
- 濕度與光照呈負相關，與時間週期特徵（如 cos_hour）呈正相關。
- 光照度與發電量之間有非常強的正向關聯，並受到時間變數的週期影響。

> 本相關性分析僅描述變數間邏輯趨勢，實際數據相關係數已省略，以遵守競賽資料保密規範。


In [ ]:
# 相關性分析與熱圖繪製（按月份）
for month in df['Month'].unique():
    month_data = df[df['Month'] == month][features]
    corr_matrix = month_data.corr()

    # 可視化邏輯
    plt.figure(figsize=(12, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, vmin=-1, vmax=1)
    plt.title(f'Month {month} - Feature Correlation Matrix')
    plt.xticks(rotation=45)
    plt.yticks(rotation=45)
    plt.show()


#按照月份建立氣候特徵模型

In [ ]:
# 訓練和儲存每月模型（此處已移除 MAE 與 R2 輸出以保護資料特性）
for month, data in monthly_data.items():
    X = data[features]
    y = data[target_columns]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=200, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # 模型效能省略顯示
    # mae = mean_absolute_error(y_test, y_pred)
    # r2 = r2_score(y_test, y_pred)

    # 儲存模型（建議使用相對路徑）
    monthly_models[month] = model
    model_filename = f"models/model_month_{month}.pkl"  # ← 改成通用目錄
    with open(model_filename, 'wb') as f:
        pickle.dump(model, f)
        # print(f"Model for month {month} saved to {model_filename}")


# **發電量模型**

還是需要環境特徵＋時間和地點做預測 準確率會比較高

In [ ]:
# 訓練每月的太陽能發電模型，使用指定氣象與地理特徵
for month, data in monthly_data.items():
    X = data[features]
    y = data[target_columns]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=200, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # 評估結果略去顯示，以保護比賽資料特性
    # mae = mean_absolute_error(y_test, y_pred)
    # r2 = r2_score(y_test, y_pred)

    # 儲存模型（使用相對路徑）
    monthly_models[month] = model
    model_filename = f"models/power_month_{month}.pkl"
    with open(model_filename, 'wb') as f:
        pickle.dump(model, f)
        # print(f"Model for month {month} saved to {model_filename}")


# 預測氣候特徵
將原始上傳資料由每10分鐘擴展為每分鐘一個預測目標

In [ ]:
import pandas as pd

# 匯入待預測的 10 分鐘間隔序號清單（不含答案）
upload_data = pd.read_csv('your_path/upload(no answer).csv')

# 擷取時間與地點代碼
upload_data['DateTime'] = upload_data['序號'].apply(lambda x: pd.to_datetime(str(x)[:12], format='%Y%m%d%H%M'))
upload_data['LocationCode'] = upload_data['序號'].apply(lambda x: int(str(x)[12:14]))

# 擴展每筆資料成 1 分鐘間隔的序號
expanded_serials = []
for _, row in upload_data.iterrows():
    start_time = row['DateTime']
    location_code = row['LocationCode']
    minute_range = pd.date_range(start=start_time, end=start_time + pd.Timedelta(minutes=9), freq='1T')
    for dt in minute_range:
        new_serial = dt.strftime('%Y%m%d%H%M') + f'{location_code:02d}'
        expanded_serials.append(new_serial)

# 儲存為 DataFrame
expanded_serials_df = pd.DataFrame(expanded_serials, columns=['序號'])

# 檢查結果格式（實際內容略）
# print(expanded_serials_df.head())  # ← 請勿公開輸出，避免洩漏資料結構


In [ ]:
upload_data = expanded_serials_df

In [ ]:
# 將預測序號或輸出結果儲存為 CSV 檔案（請依實際目錄調整路徑）
upload_data.to_csv('output/upload.csv', index=False)

透過月份模型預測之氣候特徵

In [ ]:
# 根據月份模型預測各分鐘氣候特徵（推論階段）

features = ['Hour','Minute','Day', 'LocationCode', 'sin_hour', 'cos_hour']
predictions_list = []

for month in upload_data['Month'].unique():
    month_data = upload_data[upload_data['Month'] == month]
    model_filename = f"models/weather_model/model_month_{month}.pkl"  # 相對路徑
    with open(model_filename, 'rb') as f:
        model = pickle.load(f)

    X_month = month_data[features]
    predictions = model.predict(X_month)
    predictions_df = pd.DataFrame(predictions, columns=['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)'])

    # 合併原始資料與預測結果（索引對齊）
    month_data_with_predictions = pd.concat([month_data.reset_index(drop=True), predictions_df.reset_index(drop=True)], axis=1)
    predictions_list.append(month_data_with_predictions)

# 合併所有月份預測
final_predictions = pd.concat(predictions_list, axis=0)

# 儲存結果（不輸出內容，保護資料隱私）
final_predictions.to_csv('output/weather_predictions.csv', index=False)

# 此步驟為推論階段，使用月份對應模型預測分鐘級氣象特徵。
# 所有預測結果將儲存至 output/ 目錄。
# 實際結果內容為比賽測試資料，已省略輸出顯示以保護資料內容。


In [ ]:
final_predictions
# 預測完成的結果儲存在 final_predictions
# 為避免洩漏測試資料內容，實際輸出略過顯示
# display(final_predictions.head())  # 可在本地端檢查


# 預測發電量

In [ ]:
# 載入發電量預測模型
power_model = joblib.load('models/power_model.pkl')  # 建議使用相對路徑

# 載入氣候預測結果作為發電模型輸入
final_predictions = pd.read_csv('output/weather_predictions.csv')  # 同樣使用相對路徑

# 選擇模型所需特徵欄位
power_features = ['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Hour', 'DayOfWeek']
X_input = final_predictions[power_features]

# 執行預測（不公開實際預測結果）
predicted_power = power_model.predict(X_input)

# 將預測結果加回原 DataFrame（不顯示實際資料）
final_predictions['Power(mW)'] = predicted_power

# 儲存為上傳格式
final_predictions[['序號', 'Power(mW)']].to_csv('output/upload_ready.csv', index=False)

# > 本段使用訓練好的模型預測每筆測試資料對應的太陽能發電量，並儲存為比賽規定的上傳格式。為避免洩漏測資資訊，未顯示預測內容。

In [ ]:
# 透過月份模型預測發電量
import pandas as pd
import pickle

# 載入包含氣象預測結果的資料集
final_predictions = pd.read_csv('output/weather_predictions.csv')

# 模型使用的特徵欄位
features = ['LocationCode', 'WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)']

predictions_list = []

# 根據月份（例如 7 月）做預測
for month in [7]:
    month_data = final_predictions[final_predictions['Month'] == month]
    model_filename = f"models/power_model/power_month_{month}.pkl"

    with open(model_filename, 'rb') as f:
        model = pickle.load(f)

    X_month = month_data[features]
    predictions = model.predict(X_month)

    predictions_df = pd.DataFrame(predictions, columns=['Power(mW)'])
    month_data_with_predictions = pd.concat([month_data.reset_index(drop=True), predictions_df.reset_index(drop=True)], axis=1)
    predictions_list.append(month_data_with_predictions)

# 合併結果，並儲存成上傳格式
final_predictions = pd.concat(predictions_list, axis=0)
final_predictions[['序號', 'Power(mW)']].to_csv('output/power_predictions.csv', index=False)

# 此步驟根據月份，分別載入發電模型，對每筆測試資料預測發電量 `Power(mW)`，並將結果儲存為符合比賽上傳格式的 CSV 檔案。為保護測資內容，實際輸出略。


In [ ]:
final_predictions

In [ ]:
# 將模型預測結果格式化為上傳格式，保留至小數點第二位
final_predictions['答案'] = final_predictions['power'].round(2)

# 預測資料格式為 [序號, 答案]，將其儲存為最終上傳檔案
final_predictions[['序號', '答案']].to_csv('output/upload_ready.csv', index=False)

# 為遵守比賽規則，略過實際輸出結果顯示
# display(final_predictions[['序號', '答案']].head())

In [ ]:
final_predictions.head(15)
# 預測結果包含 [序號, 預測答案]，已儲存為 CSV 檔案
# 為遵守比賽規則，以下輸出略過實際資料顯示
# final_predictions[['序號', '答案']].head()

In [ ]:
import pandas as pd

# 確保 DateTime 為 datetime 類型
final_predictions['DateTime'] = pd.to_datetime(final_predictions['DateTime'])

# 建立每 10 分鐘的區間欄位
final_predictions['10MinInterval'] = final_predictions['DateTime'].dt.floor('10T')

# 計算每 10 分鐘與地點的平均發電量
averaged_data = final_predictions.groupby(['10MinInterval', 'LocationCode'])['power'].mean().round(2).reset_index()

# 生成序號格式
averaged_data['序號'] = averaged_data['10MinInterval'].dt.strftime('%Y%m%d%H%M') + averaged_data['LocationCode'].astype(str).str.zfill(2)

# 整理成提交格式
averaged_data.rename(columns={'power': '答案'}, inplace=True)
submission_data = averaged_data[['序號', '答案']]

# 儲存為最終上傳 CSV（不顯示內容）
submission_data.to_csv('output/submission.csv', index=False)

# print(submission_data.head())  # 本地可檢查，公開時請註解


In [ ]:
submission_data

In [ ]:
# 將最終預測結果儲存為符合比賽規範的 CSV 檔案
submission_data[['序號', '答案']].to_csv('output/output_predictions.csv', index=False)